In [6]:
import pandas as pd


try:
    df = pd.read_excel('Database\Clientes1.xlsx')
    df['Classificacao'] =  df['Inatividade 12m'].apply(lambda x: 'normal' if x == 0 else ('obs' if x <=3 else 'Crítico'))
except UnicodeDecodeError:
    df = pd.read_excel('Database\Clientes1.xlsx')
    df['Classificacao'] =  df['Inatividade 12m'].apply(lambda x: 'normal' if x == 0 else ('obs' if x <=3 else 'Crítico'))


#contar quantos receberam o status de obs e crítico
print(df[df['Classificacao'] == 'Crítico']['Limite'])


#criar um novo arquivo com a coluna classificação
df.to_excel('clientes_classificados.xlsx', index=False)
print(df)





3        3313.0
12      11751.0
17      13535.0
31       2451.0
39       7882.0
         ...   
5009    24621.0
5016     2404.0
5020    34516.0
5028     1438.3
5043     1438.3
Name: Limite, Length: 304, dtype: float64
      CLIENTNUM Categoria  Idade Sexo  Dependentes           Educação  \
0     768805383   Cliente     45    M            3       Ensino Médio   
1     818770008   Cliente     49    F            5    Ensino Superior   
2     713982108   Cliente     51    M            3    Ensino Superior   
3     769911858   Cliente     40    F            4       Ensino Médio   
4     709106358   Cliente     40    M            3  Sem ensino formal   
...         ...       ...    ...  ...          ...                ...   
5058  720364158   Cliente     55    F            3    Ensino Superior   
5059  713064783   Cliente     60    F            1      Não informado   
5060  709873683   Cliente     60    M            0          Doutorado   
5061  758106108   Cliente     45    F            2  

In [8]:
# Importando as bibliotecas necessárias
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import pandas as pd

# Carregando os dados
df = pd.read_excel('Train\clientes_classificados.xlsx' )

# Verifique se as colunas existem no seu dataframe
columns = ['Idade', 'Dependentes', 'Meses como Cliente', 'Produtos Contratados', 'Inatividade 12m', 'Contatos 12m', 'Limite', 'Limite Consumido', 'Limite Disponível', 'Mudanças Transacoes_Q4_Q1', 'Valor Transacoes 12m', 'Qtde Transacoes 12m', 'Mudança Qtde Transações_Q4_Q1', 'Taxa de Utilização Cartão']
missing_columns = [col for col in columns if col not in df.columns]
if missing_columns:
    print(f"As colunas {missing_columns} não existem no dataframe. Por favor, verifique os nomes das colunas.")
else:
    # Selecionando as características que serão usadas para treinar o modelo
    features = df[columns]

    # A variável alvo é 'Classificacao'
    if 'Classificacao' in df.columns:
        target = df['Classificacao']
    else:
        print("A coluna 'Classificacao' não existe no dataframe. Por favor, verifique o nome da coluna.")

    # Dividindo os dados em conjuntos de treinamento e teste
    X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

    # Criando o modelo
    model = RandomForestClassifier(n_estimators=100)

    # Treinando o modelo
    model.fit(X_train, y_train)

    # Fazendo previsões
    predictions = model.predict(X_test)
    matriz_confusao = confusion_matrix(y_test, predictions)
    # Avaliando o modelo
    print("Acurácia:", accuracy_score(y_test, predictions))
    print("Matriz de Confusão:\n", matriz_confusao)

    # Utilizando o modelo treinado para classificar os clientes

    # Carregando a tabela clientes.csv não classificada
    new_data = pd.read_excel('Database\Clientes2.xlsx')

    # Selecionando as mesmas características que foram usadas para treinar o modelo
    new_features = new_data[columns]

    # Usando o modelo para fazer previsões nos novos dados
    new_predictions = model.predict(new_features)

    # Preencher a tabela de classificação com os novos dados
    new_data['Classificacao'] = new_predictions

#criar excel com com os dados classificados
    
    new_data.to_excel('predict\clientes_classificados_predict.xlsx', index=False)



Acurácia: 0.998025666337611
Matriz de Confusão:
 [[ 63   0   0]
 [  0   0   2]
 [  0   0 948]]


In [7]:
#graficos utilizados no artigo

import matplotlib.pyplot as plt
import seaborn as sns

# Matriz confusao 
conf_matrix = matriz_confusao
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()

# Importância das características
importances = model.feature_importances_
indices = range(len(importances))
plt.figure(figsize=(10, 7))
plt.barh(indices, importances, height=0.7)
plt.yticks(indices, columns)
plt.xlabel('Importância da característica')
plt.ylabel('Característica')
plt.title('Importância das características')
plt.show()


#grafico de barras
plt.figure(figsize=(10, 7))
sns.countplot(data=new_data, x='Classificacao')
plt.xlabel('Classificação')
plt.ylabel('Quantidade')
plt.title('Quantidade de clientes por classificação')
plt.show()



NameError: name 'confusion_matrix' is not defined